<img src="static/event-image-small.jpg" alt="drawing" width="800"/> 

# **LAB 4: RETRIEVAL-AUGMENTED GENERATION**

In this notebook, we will look at pairing a powerful LLM with an advanced
Retrieval-Augmented Generation (RAG) System. The primary and most immediate benefit
from doing so is to "ground" large language models and minize the risk of any
hallunications.

By populating the context of our LLM with specific text data identified as relevant
to the prompt, the model is more capable of being able to respond. 

Most of this notebook will leverage h2ogpte, which is a commercial product 
available from H2O. However, open source h2ogpt can also perform RAG
through it's integration with LangChain, FAISS, ChromaDB and other tools. 

Let's begin!



# Import Required Dependencies 

In [1]:
from h2ogpte import H2OGPTE
import os
import time

# Accesing Enterprise h2oGPT

Enterprise h2oGPTe is an advanced system that can intelligently process text, 
audio and image (with text) data. It's built using years of experience with
applying AI to Documents, which has now evolved into a system capable of 

As a worked out exercise, we will show how someone with a LinkedIn profile 
can download their profile as a PDF and upload it into h2ogpte.

The following screenshots show how to do so in the UI, but we will provide
programmatic examples as well.

## Step 1 - Download your profile as a PDF

---
<img src="static/linkedin-download.png" alt="drawing" width="600"/> 
---

## Step 2 - Create a new collection



---
<img src="static/new-collection1.png" alt="drawing" width="600"/> 
---

## Step 3 - Name it `MyProfile`



---
<img src="static/new-collection2.png" alt="drawing" width="600"/> 
---

## Step 4 - Upload your PDF



---
<img src="static/new-collection3.png" alt="drawing" width="400" height="600"> 
---


At this point, h2ogpte will begin indexing, chunking and storing your text data
into the vector database. 

Once complete, you will be able to begin conversing with it and other possible
interactions.

In the future, you can use the following code example to upload your data as well.

# Programmatic Upload

```python
    collection_id = h2ogpte.create_collection(name="MyProfile", description = "My Experiences")

    f = open("Profile.pdf",'rb')
    uf = h2ogpte.upload("Profile.pdf",f)
    h2ogpte.ingest_uploads(collection_id, [uf])
```

# Step 5 - Visit the UI and Generate an API Key for the Python API

Enterprise h2oGPT comes with a Python client that enables you to leverage the
power of an advanced RAG system in virtually any setting. Any application can 
now be enhanced with AI. 

In [2]:
# Authenticate
rag_url = 'https://genai-training.h2o.ai'
rag_key = 'my_key' # visit the UI for your API Key

h2ogpte = H2OGPTE(address=rag_url, api_key=rag_key)

In [3]:
# List collections
h2ogpte.list_recent_collections(0, 5)

[CollectionInfo(id='4ef8da2a-ac77-4d5b-b1e0-57c4fd9ee9b0', name='MyProfile', description='', document_count=1, document_size=58377, updated_at=datetime.datetime(2023, 11, 1, 23, 0, 10, 714633, tzinfo=datetime.timezone.utc))]

In [4]:
# Establish Collection Name
COLLECTION_NAME = 'MyProfile'
collections = h2ogpte.list_recent_collections(0, 1)

# Identify collection id
collection_id = [i.id for i in collections if i.name == COLLECTION_NAME][0]
collection_id

'4ef8da2a-ac77-4d5b-b1e0-57c4fd9ee9b0'

# What is now possible?

Behind the scenes, h2ogpte is powered by several LLMs and other models. This
enables a wide array of possible actions and fascinating ways of using
artificial intelligence for your business or application.

Let's see how well it can understand us and how it can help with career 
development.

In [5]:
queries = [
            "Whose linkedin profile are we looking at ?",
            "Where does this person work now ?",
            "How many years of experience does this user have across various different roles ? Break it down for me, with most recent employment first",
            "List the names of all the companies this person worked for ?",
            "Which company or companies did this person worked in more than 1 role ?",
            "What could be some career paths for this candidate in the next few years ?",
            "Is this person good at working at a startup or big companies ?",
            "What top 5 technical skills this person has, that would make an employer to hire immediately ?",
            "Any noteworthy soft skills ?",
            "How can we improve this person's linkedin profile ?",
            "What skills can this person learn to be successful in the next role ?"
            ]

In [7]:
for q in queries:
    print(q+"\n")
    chat_session_id = h2ogpte.create_chat_session(collection_id) # This is the collection_id we uploaded
    with h2ogpte.connect(chat_session_id) as session:
        reply = session.query(q,timeout=10600)
        answer = reply.content
        print(answer+"\n")
        #print("\nOriginal Ref:\n")
        refs = h2ogpte.list_chat_message_references(reply.id)
        if (refs):
            for ref in [refs[0]]:
                chunks = h2ogpte.get_chunks(collection_id, [ref.chunk_id])
                
                print("\n")
        else:
            print("No refs\n")

# 🎉 **CONGRATULATIONS!** You have completed this lab!
---

# 📚 **EXTENDED TASKS**

We've barely scratched the surface of evaluating Large Language Models here.

Explore what results you might get with various prompts and models 
available at gpt.h2o.ai or better yet your own hosted h2oGPT instance!

# H2O Enterprise GPT

Other notable functionality out of the box

- Embed your own question and get the similarity. 
- NLP Extraction Using LLMs
- Text Summarization API
- Choose between Lexical and Semantic Search
- Pass any open source h2oGPT Parameter
- Retrieve specific text chunks - e.g. extract the title from the first chunk
- Return the cosine similarity between two chunks
- When searching, a user can include only certain documents (parameter name is topic id)

# Open Source Alternative

<img src='static/h2oGPT.png' alt="drawing" width="200"/> 

H2O even provides RAG capabilities with open source h2oGPT and integrations
with langchain, FAISS, and ChromaDB.

For more check out the docs: https://github.com/h2oai/h2ogpt/blob/main/docs/README_CLIENT.md#gradio-client-api

Example code snippet

---
```python 
import os
from gradio_utils.grclient import GradioClient

h2ogpt_key = os.getenv('H2OGPT_KEY') or os.getenv('H2OGPT_H2OGPT_KEY')

# if you have API key for public instance:
client = GradioClient("https://gpt.h2o.ai", h2ogpt_key=h2ogpt_key)

# LLM
print(client.question("Who are you?"))

url = "https://cdn.openai.com/papers/whisper.pdf"

# Q/A
print(client.query("What is whisper?", url=url))
# summarization (map_reduce over all pages if top_k_docs=-1)
print(client.summarize("What is whisper?", url=url, top_k_docs=3))
# extraction (map per page)
print(client.extract("Give bullet for all key points", url=url, top_k_docs=3))
```
